# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

`Описание проекта` - цель данного проекта это определить регион для бурения нефтяных скважин с потенциально наибольшей прибылью и наименьшими рисками.

## Загрузка и подготовка данных

Импортируем необходимые библиотеки

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler

Загружаем данные

In [2]:
try:
    df0 = pd.read_csv('geo_data_0.csv')
    df1 = pd.read_csv('geo_data_1.csv')
    df2 = pd.read_csv('geo_data_2.csv')
except:
    df0 = pd.read_csv('/datasets/geo_data_0.csv')
    df1 = pd.read_csv('/datasets/geo_data_1.csv')
    df2 = pd.read_csv('/datasets/geo_data_2.csv')

Проводим предворительное ознакомление с данными

Выводим основную информацию для каждого региона.
Для оптимизации кода создаём функцию `show_i()` принимающую как аргумент датафрейм и выводящую на экран необходимую информацию.

In [3]:
def show_i(data):

    '''Выводим на экран следущую информацию:

    - размер таблицы
    - основную информацию
    - первые 10 строк таблицы
    - суммарная статистика по числовым параметрам
    - суммарная статистика по нечисловым параметрам'''

    display(data.shape)
    display(data.info())
    display(data.head(10))
    display(data.describe())
    display(data.describe(include=['O']))

Применяем функцию для ознакомления с данными

In [4]:
# Получение информации о регионе `geo_data_0`
show_i(df0)

(100000, 5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647
5,wX4Hy,0.969570,0.489775,-0.735383,64.741541
6,tL6pL,0.645075,0.530656,1.780266,49.055285
7,BYPU6,-0.400648,0.808337,-5.624670,72.943292
8,j9Oui,0.643105,-0.551583,2.372141,113.356160
9,OLuZU,2.173381,0.563698,9.441852,127.910945


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.500419,0.250143,2.502647,92.500000
std,0.871832,0.504433,3.248248,44.288691
min,-1.408605,-0.848218,-12.088328,0.000000
25%,-0.072580,-0.200881,0.287748,56.497507
50%,0.502360,0.250252,2.515969,91.849972
75%,1.073581,0.700646,4.715088,128.564089
max,2.362331,1.343769,16.003790,185.364347


,id
count,100000
unique,99990
top,TtcGQ
freq,2


In [5]:
# Получение информации о регионе `geo_data_1`
show_i(df1)

(100000, 5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305
5,HHckp,-3.327590,-2.205276,3.003647,84.038886
6,h5Ujo,-11.142655,-10.133399,4.002382,110.992147
7,muH9x,4.234715,-0.001354,2.004588,53.906522
8,YiRkx,13.355129,-0.332068,4.998647,134.766305
9,jG6Gi,1.069227,-11.025667,4.997844,137.945408


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,1.141296,-4.796579,2.494541,68.825000
std,8.965932,5.119872,1.703572,45.944423
min,-31.609576,-26.358598,-0.018144,0.000000
25%,-6.298551,-8.267985,1.000021,26.953261
50%,1.153055,-4.813172,2.011479,57.085625
75%,8.621015,-1.332816,3.999904,107.813044
max,29.421755,18.734063,5.019721,137.945408


,id
count,100000
unique,99996
top,wt4Uk
freq,2


In [6]:
# Получение информации о регионе `geo_data_2`
show_i(df2)

(100000, 5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746
5,LzZXx,-0.758092,0.710691,2.585887,90.222465
6,WBHRv,-0.574891,0.317727,1.773745,45.641478
7,XO8fn,-1.906649,-2.458350,-0.177097,72.480640
8,ybmQ5,1.776292,-0.279356,3.004156,106.616832
9,OilcN,-1.214452,-0.439314,5.922514,52.954532


,f0,f1,f2,product
count,100000.000000,100000.000000,100000.000000,100000.000000
mean,0.002023,-0.002081,2.495128,95.000000
std,1.732045,1.730417,3.473445,44.749921
min,-8.760004,-7.084020,-11.970335,0.000000
25%,-1.162288,-1.174820,0.130359,59.450441
50%,0.009424,-0.009482,2.484236,94.925613
75%,1.158535,1.163678,4.858794,130.595027
max,7.238262,7.844801,16.739402,190.029838


,id
count,100000
unique,99996
top,KUPhW
freq,2


Стоит отметить, что в каждом регионе есть как минимум несколько скважин пробы из которых брались дважды. Примеры таких наблюдений ниже. Пока просто отметим этот факт без каких либо выводов.

In [7]:
# находим повторные пробы нефти по скважинам
display(df0['id'].value_counts().head(11))
display(df0.loc[df0['id'] == 'AGS9W'])

TtcGQ    2
AGS9W    2
A5aEY    2
74z30    2
QcMuo    2
bxg6G    2
HZww2    2
bsk9y    2
fiKDv    2
Tdehs    2
iPtqe    1
Name: id, dtype: int64

,id,f0,f1,f2,product
42529,AGS9W,1.454747,-0.479651,0.683380,126.370504
69163,AGS9W,-0.933795,0.116194,-3.655896,19.230453


In [8]:
display(df1['id'].value_counts().head(11))
display(df1.loc[df1['id'] == 'wt4Uk'])

wt4Uk    2
LHZR0    2
bfPNe    2
5ltQ6    2
VKrsm    1
JeW0c    1
rG66O    1
Shtfk    1
LaEmn    1
cE5Zi    1
Pae1b    1
Name: id, dtype: int64

,id,f0,f1,f2,product
47591,wt4Uk,-9.091098,-8.109279,-0.002314,3.179103
82873,wt4Uk,10.259972,-9.376355,4.994297,134.766305


In [9]:
display(df2['id'].value_counts().head(11))
display(df2.loc[df2['id'] == 'VF7Jo'])

KUPhW    2
xCHr8    2
VF7Jo    2
Vcm5J    2
xYQPP    1
g209w    1
m44UK    1
JieAA    1
PKcyo    1
XJv2N    1
Y3lwp    1
Name: id, dtype: int64

,id,f0,f1,f2,product
11449,VF7Jo,2.122656,-0.858275,5.746001,181.716817
49564,VF7Jo,-0.883115,0.560537,0.723601,136.233420


`Вывод` - мы видим, что пропущенных значений нет, параметры не нуждаются в прямом кодировнии. После разделения выборок проведём масштабирование для лучшей работы модели. 
Мы обнаружили, что из некоторых скважин пробу брали дважды, данные разведанных нефтяных запасов при этом различались. Пока просто примем эту информацию к сведению.


Мы можем приступить к следующему этапу - обучению и проверке модели.

## Обучение и проверка модели

### Разбиваем данные на обучающую и валидационные выборки

Создаём функцию `split()` для разбиения данных для регионов `geo_data_0`,  `geo_data_1`,  `geo_data_2`

In [10]:
def split(dataframe):
    # извлекаем признаки, исключаем целевой признак и признак `id` который не имеет предсказательной силы тк это название скважины
    features0 = dataframe.drop(['product', 'id'], axis=1) 
    
    # извлекаем целевой признак
    target0 = dataframe['product'] 
    
    # делим на обучающую и валидационную выборки в аналогии 75 / 25
    features_train0, features_valid0, target_train0, target_valid0 = train_test_split(
    features0, target0, test_size=.25, random_state=12345)
    
    return features_train0, target_train0, features_valid0, target_valid0

Выводим размеры таблиц для региона `geo_data_0`

In [11]:
# сохряняем возвращаемые значения функции в глобальные переменные для последущего использования при обучении модели
# выводим размеры таблиц методом `shape`
features_train0 = split(df0)[0]
print(f'Выводим размер таблицы features_train0: {features_train0.shape}')
target_train0 = split(df0)[1]
print(f'Выводим размер таблицы target_train0: {target_train0.shape}')
features_valid0 = split(df0)[2]
print(f'Выводим размер таблицы features_valid0: {features_valid0.shape}')
target_valid0 = split(df0)[3]
print(f'Выводим размер таблицы target_valid0: {target_valid0.shape}')

Выводим размер таблицы features_train0: (75000, 3)
Выводим размер таблицы target_train0: (75000,)
Выводим размер таблицы features_valid0: (25000, 3)
Выводим размер таблицы target_valid0: (25000,)


Выводим размеры таблиц для региона `geo_data_1`

In [12]:
# сохряняем возвращаемые значения функции в глобальные переменные для последущего использования при обучении модели
# выводим размеры таблиц методом `shape`
features_train1 = split(df1)[0]
print(f'Выводим размер таблицы features_train1: {features_train1.shape}')
target_train1 = split(df1)[1]
print(f'Выводим размер таблицы target_train1: {target_train1.shape}')
features_valid1 = split(df1)[2]
print(f'Выводим размер таблицы features_valid1: {features_valid1.shape}')
target_valid1 = split(df1)[3]
print(f'Выводим размер таблицы target_valid1: {target_valid1.shape}')

Выводим размер таблицы features_train1: (75000, 3)
Выводим размер таблицы target_train1: (75000,)
Выводим размер таблицы features_valid1: (25000, 3)
Выводим размер таблицы target_valid1: (25000,)


Выводим размеры таблиц для региона `geo_data_2`

In [13]:
# сохряняем возвращаемые значения функции в глобальные переменные для последущего использования при обучении модели
# выводим размеры таблиц методом `shape`
features_train2 = split(df2)[0]
print(f'Выводим размер таблицы features_train2: {features_train2.shape}')
target_train2 = split(df2)[1]
print(f'Выводим размер таблицы target_train2: {target_train2.shape}')
features_valid2 = split(df2)[2]
print(f'Выводим размер таблицы features_valid2: {features_valid2.shape}')
target_valid2 = split(df2)[3]
print(f'Выводим размер таблицы target_valid2: {target_valid2.shape}')

Выводим размер таблицы features_train2: (75000, 3)
Выводим размер таблицы target_train2: (75000,)
Выводим размер таблицы features_valid2: (25000, 3)
Выводим размер таблицы target_valid2: (25000,)


Проводим масштабирование данных для лучшей работы модели линейной регрессии

In [14]:
pd.options.mode.chained_assignment = None
numeric = ['f0', 'f1', 'f2']

# df0 scaling
scaler = StandardScaler()
scaler.fit(df0[numeric])

features_train0[numeric] = scaler.transform(features_train0[numeric])
features_valid0[numeric] = scaler.transform(features_valid0[numeric])

# df1 scaling
scaler = StandardScaler()
scaler.fit(df1[numeric])

features_train1[numeric] = scaler.transform(features_train1[numeric])
features_valid1[numeric] = scaler.transform(features_valid1[numeric])

# df2 scaling
scaler = StandardScaler()
scaler.fit(df2[numeric])

features_train2[numeric] = scaler.transform(features_train2[numeric])
features_valid2[numeric] = scaler.transform(features_valid2[numeric])


### Обучаем модели при помощи линейной регрессии

Приступаем к обучению модели при помощи линейной регресси так как остальные модели, в данном случае, недостаточно предсказуемые.

Проводим обучение на выборке для региона `geo_data_0`

In [15]:
# инициализируем модель LinearRegression
model = LinearRegression()

# обучаем модель на тренировочной выборке
model.fit(features_train0, target_train0, sample_weight=None)

# получаем предсказания модели на валидационной выборке
predictions_valid0 = model.predict(features_valid0) 

# сохраняем результаты предсказания в валидационной выборке вместе с правильными ответами и выводим на экран
features_valid0['predictions_valid0'] = predictions_valid0
features_valid0['target_valid0'] = target_valid0
display(features_valid0.head(3))

# средний запас предсказанного сырья в регионе
predictions_valid0_mean = features_valid0["predictions_valid0"].mean()
print(f'Средний предсказанный запас сырья в регионе geo_data_0: {predictions_valid0_mean} (тыс. баррелей)')

# считаем значение метрики RMSE на валидационной выборке
result0 = mean_squared_error(target_valid0, predictions_valid0) ** 0.5 
print("RMSE модели линейной регрессии на валидационной выборке features_train0 (регион `geo_data_0`)", result0)


,f0,f1,f2,predictions_valid0,target_valid0
71751,0.514495,-0.609975,-0.125274,95.894952,10.038645
80493,0.564968,-0.086181,-0.814266,77.572583,114.551489
2655,0.802263,-1.612709,-1.340926,77.892640,132.603635


Средний предсказанный запас сырья в регионе geo_data_0: 92.59256778438035 (тыс. баррелей)
RMSE модели линейной регрессии на валидационной выборке features_train0 (регион `geo_data_0`) 37.5794217150813


Проводим обучение на выборке для региона `geo_data_1`

In [16]:
# обучаем модель на тренировочной выборке
model.fit(features_train1, target_train1, sample_weight=None)

# получаем предсказания модели на валидационной выборке
predictions_valid1 = model.predict(features_valid1) 

# сохраняем результаты предсказания в валидационной выборке вместе с правильными ответами и выводим на экран
features_valid1['predictions_valid1'] = predictions_valid1
features_valid1['target_valid1'] = target_valid1
display(features_valid1.head(3))

# средний запас предсказанного сырья в регионе
predictions_valid1_mean = features_valid1["predictions_valid1"].mean()
print(f'Средний предсказанный запас сырья в регионе geo_data_1: {predictions_valid1_mean} (тыс. баррелей)')

# считаем значение метрики RMSE на валидационной выборке
result1 = mean_squared_error(target_valid1, predictions_valid1) ** 0.5 
print("RMSE модели линейной регрессии на валидационной выборке features_train1 (регион `geo_data_1`)", result1)

,f0,f1,f2,predictions_valid1,target_valid1
71751,-0.168769,0.573081,0.298004,82.663314,80.859783
80493,0.878198,-1.774443,-0.293020,54.431786,53.906522
2655,-0.853107,-0.004076,-0.875278,29.748760,30.132364


Средний предсказанный запас сырья в регионе geo_data_1: 68.72854689544602 (тыс. баррелей)
RMSE модели линейной регрессии на валидационной выборке features_train1 (регион `geo_data_1`) 0.893099286775617


Проводим обучение на выборке для региона `geo_data_2`

In [17]:
#model = LinearRegression()

# обучаем модель на тренировочной выборке
model.fit(features_train2, target_train2, sample_weight=None)

# получаем предсказания модели на валидационной выборке
predictions_valid2 = model.predict(features_valid2) 

# сохраняем результаты предсказания в валидационной выборке вместе с правильными ответами и выводим на экран
features_valid2['predictions_valid2'] = predictions_valid2
features_valid2['target_valid2'] = target_valid2
display(features_valid2.head(3))

# средний запас предсказанного сырья в регионе
predictions_valid2_mean = features_valid2["predictions_valid2"].mean()
print(f'Средний предсказанный запас сырья в регионе geo_data_2: {predictions_valid2_mean} (тыс. баррелей)')

# считаем значение метрики RMSE на валидационной выборке
result2 = mean_squared_error(target_valid2, predictions_valid2) ** 0.5 
print("RMSE модели линейной регрессии на валидационной выборке features_train2 (регион `geo_data_2`)", result2)

,f0,f1,f2,predictions_valid2,target_valid2
71751,-0.835282,-2.230409,-0.077538,93.599633,61.212375
80493,-0.820214,0.738915,-0.999463,75.105159,41.850118
2655,-2.649871,-0.237585,-0.243523,90.066809,57.776581


Средний предсказанный запас сырья в регионе geo_data_2: 94.96504596800489 (тыс. баррелей)
RMSE модели линейной регрессии на валидационной выборке features_train2 (регион `geo_data_2`) 40.02970873393434


Итоговые показатели:

In [18]:
# итоговые средние предсказанные запасы нефти по регионам
print(f'Предсказанный запаса сырья в регионе geo_data_0: {predictions_valid0_mean: .2f} (тыс. баррелей)')
print(f'RMSE geo_data_0: {result0: .2f}')
print()
print(f'Предсказанный запаса сырья в регионе geo_data_1: {predictions_valid1_mean: .2f} (тыс. баррелей)')
print(f'RMSE geo_data_1: {result1: .2f} ')
print()
print(f'Предсказанный запаса сырья в регионе geo_data_2: {predictions_valid2_mean: .2f} (тыс. баррелей)')
print(f'RMSE geo_data_2: {result2: .2f}')

Предсказанный запаса сырья в регионе geo_data_0:  92.59 (тыс. баррелей)
RMSE geo_data_0:  37.58

Предсказанный запаса сырья в регионе geo_data_1:  68.73 (тыс. баррелей)
RMSE geo_data_1:  0.89 

Предсказанный запаса сырья в регионе geo_data_2:  94.97 (тыс. баррелей)
RMSE geo_data_2:  40.03


`Вывод` - мы видим, самые высокие показатели залежей сырья предсказаны для районов `geo_data_0` и `geo_data_2` (92.59 и 94.97 тыс. баррелей соответственно). 

Однако, данные по этим регионам предсказанны с относительно высокой среднеквадратичной ошибкой, что позволяет нам предположить что реальный разброс запасов сырья может значительно отличаться от предсказанного нашей моделью.

В то же время, предсказания для региона `geo_data_1` отличаются значитеьно более высокими метриками точности. RMSE = 0.89.
Это говорит нам о том что точность предказанных залежей сырья было предсказанно с более высокой долей вероятности.

## Подготовка к расчёту прибыли

### Сохряняем ключевые значения для расчёта прибыли в переменных и высчитываем минимальный объём сырья в скважине для получения прибыли

In [19]:
# бюджет на каждый регион (руб.)
BUDGET = 10000000000

# доход с 1К баррелей нефти (руб.)
REVENUE_1K_BAR = 450000

In [20]:
# высчитываем объём сырья в регионе для выхода в прибыль
profit_margin = (BUDGET / REVENUE_1K_BAR) / 200
print(f'{profit_margin: .2f} тыс. баррелей нефти в скважине необходимо для выхода в прибыль')

 111.11 тыс. баррелей нефти в скважине необходимо для выхода в прибыль


In [21]:
print('Средний запас в регионе `geo_data_0`: ', target_valid0.mean())
print('Средний запас в регионе `geo_data_1`: ', target_valid1.mean())
print('Средний запас в регионе `geo_data_1`: ', target_valid2.mean())

Средний запас в регионе `geo_data_0`:  92.07859674082927
Средний запас в регионе `geo_data_1`:  68.72313602435997
Средний запас в регионе `geo_data_1`:  94.88423280885438


`Вывод` - мы видим, что средний объём нефти в скважине необходимый для окупаемости добычи - выше чем средние показатели запасов сырья в трёх регионах. Это говорит о том, что для окупаемости разработки необходимо отобрать самые плодородные скважины которые превышают порог рентабильности добычи в  111.11 тыс. баррелей нефти.

## Расчёт прибыли и рисков 

Переводим array в Series для дальнейших калькуляций

In [22]:
# предсказания принадлежат к типу array и нуждаются в переводе в Series
print(type(predictions_valid0))
print(type(predictions_valid1))
print(type(predictions_valid2))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [23]:
# целевые признаки не нуждается в переводе в Seies
print(type(target_valid0))
print(type(target_valid1))
print(type(target_valid2))

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


Переводим предсказания из array в Series и сбрасываем индексы на целевом признаке и предсказаниях с помощью функции.

In [24]:
# делаем функцию для сброса индексов на целевом признаке и предсказаниях, а также переводим предсказания из array в Series
def reset_i(predictions, target):
    predictions_series = pd.Series(predictions)
    predictions_reset = predictions_series.reset_index(drop=True)
    target_reset = target.reset_index(drop=True)
    return predictions_reset, target_reset

Применяем функцию `reset_i` для смены типа прогнозов на Series и сброса индексов

In [25]:
predictions_valid00, target_valid00 = reset_i(predictions_valid0, target_valid0)
predictions_valid01, target_valid01 = reset_i(predictions_valid1, target_valid1)
predictions_valid02, target_valid02 = reset_i(predictions_valid2, target_valid2)

Создаём функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели

In [26]:
# на вход принимаем предсказания модели, целевые признаки, кол-во вышек, доход в 1К бар. и бюджет
def revenue(predictions, target, well_count):
    '''
    - сортируем предсказания
    - отбираем самые большие значения
    - считаем доход
    '''
    predicted_sorted = predictions.sort_values(ascending=False)
    selected_target = target[predicted_sorted.index][:well_count]
    revenue_selected = selected_target.sum() * REVENUE_1K_BAR - BUDGET
    return revenue_selected

Созраняем значения прибыли по регионам в переменные для дальнейшего использования и выводим их на экран

In [27]:
region_1 = int(revenue(predictions_valid00, target_valid00, 200))
region_2 = int(revenue(predictions_valid01, target_valid01, 200))
region_3 = int(revenue(predictions_valid02, target_valid02, 200))

In [28]:
print(f'1 регион {region_1} тыс. баррелей')
print(f'2 регион {region_1} тыс. баррелей')
print(f'3 регион {region_3} тыс. баррелей')

1 регион 3320826043 тыс. баррелей
2 регион 3320826043 тыс. баррелей
3 регион 2710349963 тыс. баррелей


Для наглядности, создадим таблицу с основными показателями по региону

In [29]:
# создаём новый датафрейм
revenue_df = pd.DataFrame({ 'Регион': ['Регион 1', 'Регион 2', 'Регион 3'], 
'Средняя добыча региона (тыс. баррелей)': [region_1, region_2, region_3],
'Средняя добыча скважины (тыс. баррелей)': [target_valid0.mean(), target_valid1.mean(), target_valid2.mean()]
})

# меняем индекс колонку на 'Регион'
revenue_df = revenue_df.set_index('Регион')

# сбрасываем старые индексы
revenue_df = revenue_df.reset_index()

# выводим таблицу на экран
revenue_df

,Регион,Средняя добыча региона (тыс. баррелей),Средняя добыча скважины (тыс. баррелей)
0,Регион 1,3320826043,92.078597
1,Регион 2,2415086696,68.723136
2,Регион 3,2710349963,94.884233


### Находим распределение прибыли

Применяем технику Bootstrap с 1000 выборок, чтобы найти распределение прибыли - пишем для этого функцию `profit_distr()`

In [30]:
def profit_distr(target, predictions):
    state = np.random.RandomState(12345)
    values = []
    # делаем butstrep калькуляцию
    for i in range(1000):
        target_subsample = target.sample(n=500, replace=True, random_state=state)
        probs_subsample = predictions[target_subsample.index]
        value = revenue(probs_subsample, target_subsample, 200)
        values.append(value)

    # список со значениями выручки
    values = pd.Series(values)

    # Квантиль 2,5%
    lower = values.quantile(0.025)
    # Квантиль 97,5%
    upper = values.quantile(0.975)

    # считаем среднюю выручку
    mean = values.mean()

    # считаем риски получить убыток (отрицательное) значение прибыли
    risk = (((values < 0).sum()) / values.count()) * 100

    return mean, risk, lower, upper  



Применяем функцию для подсчёта по всем трём регионам по-очереди. Сохраняем каждую новую величину в переменной для последующего вывода на экран в виде таблицы.

In [31]:
# регион 1
# сохраняем результаты вывода функции в переменных

region_0_mean, region_0_risk, region_0_lower, region_0_upper = profit_distr(target_valid00, predictions_valid00)

In [32]:
# регион 2
# сохраняем результаты вывода функции в переменныХ

region_1_mean, region_1_risk, region_1_lower, region_1_upper = profit_distr(target_valid01, predictions_valid01)

In [33]:
# регион 3
# сохраняем результаты вывода функции в переменных

region_2_mean, region_2_risk, region_2_lower, region_2_upper = profit_distr(target_valid02, predictions_valid02)

Создаём датафрейм с ключевыми показателями

In [34]:
final_table = pd.DataFrame({
    'Регион': ['Регион 1', 'Регион 2', 'Регион 3'],
    'Средняя добыча региона (тыс. баррелей)': [f'{region_1}', f'{region_2}', f'{region_3}'],
    'Средняя выручка (рублей)' : [f'{region_0_mean:.0f}', f'{region_1_mean:.0f}', f'{region_2_mean:.0f}'],
    'Риск получения убытка (%):' : [f'{region_0_risk:.0f}', f'{region_1_risk:.0f}', f'{region_2_risk:.0f}'],
    '2.5%-квантиль (рублей)': [f'{region_0_lower:.0f}', f'{region_1_lower:.0f}', f'{region_2_lower:.0f}'],
    '97.5%-квантиль (рублей)' : [f'{region_0_upper:.0f}', f'{region_1_upper:.0f}', f'{region_2_upper:.0f}']
        
})

Выводим финальную таблицу на экран.

In [35]:
display(final_table)

,Регион,Средняя добыча региона (тыс. баррелей),Средняя выручка (рублей),Риск получения убытка (%):,2.5%-квантиль (рублей),97.5%-квантиль (рублей)
0,Регион 1,3320826043,425938527,6,-102090095,947976353
1,Регион 2,2415086696,515222773,1,68873225,931547591
2,Регион 3,2710349963,435008363,6,-128880547,969706954


`Вывод` - В результате проведённого исследования, построения модели и оценки рисков, мы пришли в выводу что саммым предпочтительным регионом для добычи является `Регион 2` потому что имеет наибольшую среднюю выручку (515.222.773,00 руб.) и минимальные риски ухода в убыток - всего 1%.

Доверительный интервал прибыли вторго региона явно демонстрирует лучшие показатели по прибыли включая только положительный доход.

`Итого:` несмотря на наименьшую среднюю добычу второго региона (2.415.086.696 тыс. баррелей), даный регион считаю более перспективным в плане доходности и минимизации рисков.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [x]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [x]  Выполнен шаг 4: посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован